In [1]:
#Imports required packages
import pandas as pd
import numpy as np
from numpy import mean
import sklearn
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import cross_validate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
import gensim
import time
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
#from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score, precision_score
from autosklearn.metrics import (accuracy, f1,roc_auc, precision, average_precision,recall)
import autosklearn.classification
import PipelineProfiler
import time
import psutil

In [2]:
# load data - description and title files
IR_titles = pd.read_csv('title.csv')
IR_desc = pd.read_csv('desc.csv')

In [4]:
IR_desc.iloc[:, 2].head(2)

0    there coupl place your catch interruptedioexce...
1    the execut method ha follow simplifi flow 1 ge...
Name: the httpstate class ha clearcooki method not synchron but should consid modifi arraylist which unsynchron all other method which modifi read arraylist synchron except clearcooki method I stumbl upon fact becaus webapp I am work use httpclient threw illegalargumentexcept indic one cooki array return methodnam null which shouldnt possibl upon further inspect and test onli possibl option threadsafeti hole left unsynchron clearcooki method caus issu, dtype: object

In [5]:
traindf = pd.DataFrame()
traindf['labels'] = IR_desc.iloc[:, 1] #labels: 1: Bug and 0; Other
traindf['Summary'] = IR_titles.iloc[:, 2]+" "+IR_desc.iloc[:, 2]

In [6]:
traindf.head(2)

,labels,Summary
0,1,catch sockettimeoutexcept not interruptedioexc...
1,1,except dure writerequest leav connect unreleas...


In [7]:
# checking the shape of the data
traindf.shape #,testdf.shape

(5590, 2)

In [8]:
# defining a function to clean data
def preprocess(text):  
    text = str(text)
    
    # lowercase
    text = text.lower()
    
    # remove non_alphanum
    #text = gensim.parsing.preprocessing.strip_non_alphanum(text)
    
    # remove html tags
    #text = gensim.parsing.preprocessing.strip_tags(text)
  
    # remove punctuation
    text = gensim.parsing.preprocessing.strip_punctuation(text)
  
    # remove numerics
    #text = gensim.parsing.preprocessing.strip_numeric(text)
  
    # remove consecutive whitespace characters and convert tabs to spaces
    text = gensim.parsing.preprocessing.strip_multiple_whitespaces(text)
  
    #text = gensim.parsing.preprocessing.strip_short(text, minsize=3)
  
    #remove stop-words
    text = gensim.parsing.preprocessing.remove_stopwords(text)
    
    # make stems
    text = gensim.parsing.preprocessing.stem_text(text)
    
    return text
    

In [9]:
# timer start
start_time = time.time()
# clean training data
for index, row in traindf.iterrows():
    text = row['Summary']
    text = preprocess(text)
    traindf.at[index, 'Summary'] = text

In [10]:
traindf.head(2)

,labels,Summary
0,1,catch sockettimeoutexcept interruptedioexcept ...
1,1,dure writerequest leav connect unrelea execut ...


In [11]:
#get actual labels of training data
y_train = traindf['labels'].values

X_train = traindf[['Summary']].copy()

In [12]:
# Create an AutoML classifier
automl_classifier = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=90*60,
    resampling_strategy="cv",
    resampling_strategy_arguments={"folds": 10},
    metric=roc_auc,
    scoring_functions=[roc_auc, average_precision, accuracy, f1, precision, recall]
)

In [13]:
automl_classifier.fit(X_train, y_train)

/home/laiq/.local/lib/python3.8/site-packages/autosklearn/data/feature_validator.py:318: UserWarning: Input Column Summary has generic type object. Autosklearn will treat this column as string. Please ensure that this setting is suitable for your task.
  warnings.warn(


AutoSklearnClassifier(ensemble_class=<class 'autosklearn.ensembles.ensemble_selection.EnsembleSelection'>,
                      metric=roc_auc, per_run_time_limit=540,
                      resampling_strategy='cv',
                      resampling_strategy_arguments={'folds': 10},
                      scoring_functions=[roc_auc, average_precision, accuracy,
                                         f1, precision, recall],
                      time_left_for_this_task=5400)

In [14]:
perfor_over_time =automl_classifier.performance_over_time_
perfor_over_time.to_csv('perfor_over_time_90mintues_cv_scores.csv', index=False) 
perfor_over_time

,Timestamp,ensemble_optimization_score,single_best_optimization_score,single_best_train_score
23,2023-10-11 11:37:29.000000,NaN,0.833478,0.999199
22,2023-10-11 11:37:29.000000,NaN,0.833478,0.999199
0,2023-10-11 11:37:29.507224,0.833221,0.833478,0.999199
1,2023-10-11 11:41:39.947203,0.833764,0.833478,0.999199
29,2023-10-11 11:52:34.000000,0.833764,0.858286,0.878606
28,2023-10-11 11:52:34.000000,0.833764,0.858286,0.878606
27,2023-10-11 11:52:34.000000,0.833764,0.858286,0.878606
26,2023-10-11 11:52:34.000000,0.833764,0.858286,0.878606
25,2023-10-11 11:52:34.000000,0.833764,0.858286,0.878606
24,2023-10-11 11:52:34.000000,0.833764,0.858286,0.878606


In [15]:
print(automl_classifier.sprint_statistics())

auto-sklearn results:
  Dataset name: 1763f644-6819-11ee-80c0-747827e65785
  Metric: roc_auc
  Best validation score: 0.874987
  Number of target algorithm runs: 29
  Number of successful target algorithm runs: 22
  Number of crashed target algorithm runs: 3
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 0



In [16]:
# print training time
#print("total time:", time.time() - start_time)

total time: 5417.731060266495


In [17]:
pid = psutil.Process().pid
memory_usage_in_bytes = psutil.Process(pid).memory_info().rss
memory_usage_in_megabytes = memory_usage_in_bytes / 1024**2

print(memory_usage_in_megabytes)

6557.08984375


In [18]:
print(automl_classifier.leaderboard())

          rank  ensemble_weight               type      cost    duration
model_id                                                                
25           1             0.32                mlp  0.125013  214.286597
22           2             0.22                mlp  0.125628  110.059520
21           3             0.06                mlp  0.142288  148.786433
16           4             0.10        extra_trees  0.150311  124.004522
10           5             0.04  gradient_boosting  0.150701  180.722810
13           6             0.06        extra_trees  0.158342  106.644444
12           7             0.02      random_forest  0.161851  144.406035
2            8             0.04      random_forest  0.166522  292.258490
15           9             0.02                mlp  0.167633  430.430335
20          10             0.02  gradient_boosting  0.168899  104.449907
26          11             0.02  gradient_boosting  0.176295  142.551524
23          12             0.08         libsvm_svc 

In [19]:
automl_classifier.leaderboard(detailed = True, ensemble_only=False)

,rank,ensemble_weight,type,cost,duration,config_id,train_loss,seed,start_time,end_time,budget,status,data_preprocessors,feature_preprocessors,balancing_strategy,config_origin
model_id,,,,,,,,,,,,,,,,
25,1,0.32,mlp,0.125013,214.286597,24,0.077751,0,1.697021e+09,1.697022e+09,0.0,StatusType.SUCCESS,[],[fast_ica],weighting,Initial design
22,2,0.22,mlp,0.125628,110.059520,21,0.094214,0,1.697021e+09,1.697021e+09,0.0,StatusType.SUCCESS,[],[feature_agglomeration],none,Initial design
19,3,0.00,mlp,0.132839,127.156641,18,0.121940,0,1.697021e+09,1.697021e+09,0.0,StatusType.SUCCESS,[],[feature_agglomeration],weighting,Initial design
5,4,0.00,libsvm_svc,0.141714,113.177651,4,0.121394,0,1.697018e+09,1.697018e+09,0.0,StatusType.SUCCESS,[],[feature_agglomeration],weighting,Initial design
21,5,0.06,mlp,0.142288,148.786433,20,0.087629,0,1.697021e+09,1.697021e+09,0.0,StatusType.SUCCESS,[],[liblinear_svc_preprocessor],weighting,Initial design
24,6,0.00,mlp,0.143801,117.881360,23,0.134924,0,1.697021e+09,1.697021e+09,0.0,StatusType.SUCCESS,[],[feature_agglomeration],none,Initial design
16,7,0.10,extra_trees,0.150311,124.004522,15,0.030534,0,1.697020e+09,1.697020e+09,0.0,StatusType.SUCCESS,[],[fast_ica],none,Initial design
10,8,0.04,gradient_boosting,0.150701,180.722810,9,0.004227,0,1.697019e+09,1.697019e+09,0.0,StatusType.SUCCESS,[],[feature_agglomeration],none,Initial design
7,9,0.00,sgd,0.150982,92.675042,6,0.140851,0,1.697018e+09,1.697018e+09,0.0,StatusType.SUCCESS,[],[select_percentile_classification],none,Initial design


In [20]:
df_leaderboard = pd.DataFrame(automl_classifier.leaderboard(detailed = True, ensemble_only=False)).sort_values(by = 'rank', ascending = True)
df_leaderboard.to_csv('iter_1_90mintues_leaderboard.csv', index=False) 
df_leaderboard

,rank,ensemble_weight,type,cost,duration,config_id,train_loss,seed,start_time,end_time,budget,status,data_preprocessors,feature_preprocessors,balancing_strategy,config_origin
model_id,,,,,,,,,,,,,,,,
25,1,0.32,mlp,0.125013,214.286597,24,0.077751,0,1.697021e+09,1.697022e+09,0.0,StatusType.SUCCESS,[],[fast_ica],weighting,Initial design
22,2,0.22,mlp,0.125628,110.059520,21,0.094214,0,1.697021e+09,1.697021e+09,0.0,StatusType.SUCCESS,[],[feature_agglomeration],none,Initial design
19,3,0.00,mlp,0.132839,127.156641,18,0.121940,0,1.697021e+09,1.697021e+09,0.0,StatusType.SUCCESS,[],[feature_agglomeration],weighting,Initial design
5,4,0.00,libsvm_svc,0.141714,113.177651,4,0.121394,0,1.697018e+09,1.697018e+09,0.0,StatusType.SUCCESS,[],[feature_agglomeration],weighting,Initial design
21,5,0.06,mlp,0.142288,148.786433,20,0.087629,0,1.697021e+09,1.697021e+09,0.0,StatusType.SUCCESS,[],[liblinear_svc_preprocessor],weighting,Initial design
24,6,0.00,mlp,0.143801,117.881360,23,0.134924,0,1.697021e+09,1.697021e+09,0.0,StatusType.SUCCESS,[],[feature_agglomeration],none,Initial design
16,7,0.10,extra_trees,0.150311,124.004522,15,0.030534,0,1.697020e+09,1.697020e+09,0.0,StatusType.SUCCESS,[],[fast_ica],none,Initial design
10,8,0.04,gradient_boosting,0.150701,180.722810,9,0.004227,0,1.697019e+09,1.697019e+09,0.0,StatusType.SUCCESS,[],[feature_agglomeration],none,Initial design
7,9,0.00,sgd,0.150982,92.675042,6,0.140851,0,1.697018e+09,1.697018e+09,0.0,StatusType.SUCCESS,[],[select_percentile_classification],none,Initial design


In [21]:
#automl_classifier.sprint_statistics()

In [22]:
df_cv_results = pd.DataFrame(automl_classifier.cv_results_).sort_values(by = 'mean_test_score', ascending = False)
df_cv_results

,mean_test_score,rank_test_scores,metric_roc_auc,metric_average_precision,metric_accuracy,metric_f1,metric_precision,metric_recall,mean_fit_time,params,...,param_feature_preprocessor:fast_ica:n_components,param_feature_preprocessor:kernel_pca:coef0,param_feature_preprocessor:kernel_pca:degree,param_feature_preprocessor:kernel_pca:gamma,param_feature_preprocessor:nystroem_sampler:coef0,param_feature_preprocessor:nystroem_sampler:degree,param_feature_preprocessor:nystroem_sampler:gamma,param_feature_preprocessor:select_rates_classification:mode,param_data_preprocessor:feature_type:text_transformer:text_encoding:tfidf_encoding:ngram_range_char,param_data_preprocessor:feature_type:text_transformer:text_encoding:tfidf_encoding:ngram_range_word
23,0.874987,1,0.874987,0.778326,0.801431,0.695603,0.742925,0.655505,214.286597,"{'balancing:strategy': 'weighting', 'classifie...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
20,0.874372,2,0.874372,0.776669,0.803041,0.703204,0.736934,0.673519,110.059520,"{'balancing:strategy': 'none', 'classifier:__c...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
17,0.867161,3,0.867161,0.766240,0.793023,0.672831,0.744395,0.614227,127.156641,"{'balancing:strategy': 'weighting', 'classifie...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
3,0.858286,4,0.858286,0.749672,0.782826,0.702205,0.669354,0.739047,113.177651,"{'balancing:strategy': 'weighting', 'classifie...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
19,0.857712,5,0.857712,0.757930,0.792487,0.688300,0.718928,0.660646,148.786433,"{'balancing:strategy': 'weighting', 'classifie...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
22,0.856199,6,0.856199,0.749571,0.784973,0.658320,0.733373,0.597727,117.881360,"{'balancing:strategy': 'none', 'classifier:__c...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
14,0.849689,7,0.849689,0.744585,0.715385,0.342593,0.860457,0.214542,124.004522,"{'balancing:strategy': 'none', 'classifier:__c...",...,706.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
8,0.849299,8,0.849299,0.749694,0.784436,0.653735,0.737382,0.587415,180.722810,"{'balancing:strategy': 'none', 'classifier:__c...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
5,0.849018,9,0.849018,0.743497,0.777996,0.643931,0.726312,0.578652,92.675042,"{'balancing:strategy': 'none', 'classifier:__c...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
11,0.841658,10,0.841658,0.729147,0.653131,0.000000,0.000000,0.000000,106.644444,"{'balancing:strategy': 'none', 'classifier:__c...",...,98.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN


In [23]:
df_cv_results.to_csv('iter_1_90mintues_cv_scores.csv', index=False) 

In [24]:
automl_classifier.get_models_with_weights()

[(0.32,
  SimpleClassificationPipeline({'balancing:strategy': 'weighting', 'classifier:__choice__': 'mlp', 'data_preprocessor:__choice__': 'feature_type', 'feature_preprocessor:__choice__': 'fast_ica', 'classifier:mlp:activation': 'tanh', 'classifier:mlp:alpha': 2.316980754198592e-05, 'classifier:mlp:batch_size': 'auto', 'classifier:mlp:beta_1': 0.9, 'classifier:mlp:beta_2': 0.999, 'classifier:mlp:early_stopping': 'train', 'classifier:mlp:epsilon': 1e-08, 'classifier:mlp:hidden_layer_depth': 1, 'classifier:mlp:learning_rate_init': 0.0027947098669343414, 'classifier:mlp:n_iter_no_change': 32, 'classifier:mlp:num_nodes_per_layer': 30, 'classifier:mlp:shuffle': 'True', 'classifier:mlp:solver': 'adam', 'classifier:mlp:tol': 0.0001, 'data_preprocessor:feature_type:text_transformer:text_encoding:__choice__': 'tfidf_encoding', 'data_preprocessor:feature_type:text_transformer:text_feature_reduction:n_components': 100, 'feature_preprocessor:fast_ica:algorithm': 'deflation', 'feature_preprocesso

In [ ]:
# automl is an object Which has already been created.
profiler_data= PipelineProfiler.import_autosklearn(automl_classifier)
PipelineProfiler.plot_pipeline_matrix(profiler_data)